In [2]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from datetime import datetime
import os

In [8]:
# Cargar el modelo entrenado
model = load_model('../models/reconocimiento_facial_15_personas.h5')

# Clase de nombres de las 15 personas
class_names = ['David']  # Cambiar según corresponda

# Inicializar la cámara web
cap = cv2.VideoCapture(0)

# Cargar Haar Cascade para la detección de rostros
face_cascade = cv2.CascadeClassifier('../models/haarcascade_frontalface_alt.xml')

# Crear el archivo .csv si no existe
csv_file = '../results/asistencia.csv'

if not os.path.exists(csv_file):
    # Crear un DataFrame vacío con columnas "Nombre" y "Fecha"
    df = pd.DataFrame(columns=["Nombre", "Fecha"])
    df.to_csv(csv_file, index=False)

In [9]:
# Función para registrar la asistencia en el archivo .csv
def registrar_asistencia(nombre_persona):
    # Obtener la fecha y hora actual
    fecha_actual = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Crear una fila con el nombre de la persona y la fecha actual
    nueva_fila = {'Nombre': nombre_persona, 'Fecha': fecha_actual}
    
    # Leer el archivo .csv existente
    df = pd.read_csv(csv_file)
    
    # Verificar si la persona ya ha sido registrada en la sesión actual
    if not ((df['Nombre'] == nombre_persona) & (df['Fecha'].str.startswith(datetime.now().strftime('%Y-%m-%d')))).any():
        # Agregar la nueva fila al archivo .csv
        df = df.append(nueva_fila, ignore_index=True)
        df.to_csv(csv_file, index=False)
        print(f'Asistencia registrada para {nombre_persona} a las {fecha_actual}')
    else:
        print(f'{nombre_persona} ya está registrada hoy.')

In [10]:
while True:
    ret, frame = cap.read()
    
    if not ret:
        print("No se puede acceder a la cámara")
        break
    
    # Convertir a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detectar rostros
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Procesar cada rostro detectado
    for (x, y, w, h) in faces:
        # Dibujar un rectángulo alrededor del rostro
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Recortar el rostro de la imagen original
        face = frame[y:y+h, x:x+w]
        
        # Preprocesar el rostro
        face_resized = cv2.resize(face, (150, 150))  # Ajustar al tamaño que espera el modelo
        face_array = image.img_to_array(face_resized)
        face_array = np.expand_dims(face_array, axis=0)  # Añadir una dimensión para el lote
        
        # Normalizar el valor de los píxeles si es necesario (esto depende del preprocesamiento utilizado en el modelo)
        face_array /= 255.0
        
        # Realizar predicción
        predictions = model.predict(face_array)
        
        # Obtener el índice de la clase con la mayor probabilidad
        predicted_class = np.argmax(predictions)
        
        # Obtener el nombre de la persona
        person_name = class_names[predicted_class]
        
        # Mostrar el nombre sobre el rostro en el video
        cv2.putText(frame, person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Registrar la asistencia de la persona
        registrar_asistencia(person_name)
    
    # Mostrar el video en tiempo real
    cv2.imshow('Reconocimiento Facial en Tiempo Real', frame)
    
    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()

ValueError: Input 0 of layer "functional_4" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(1, 150, 150, 3)

: 